In [1]:
%config InlineBackend.figure_format='retina'

# Add the parent directory to the path so that we can import the modules.
from functools import partial
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from market_conditions import *
from dynamic_contagion import *
from reinsurance import *

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Other parameters
maturity = 3
markup = 0.4
R = int(1e6)
seed = 123

# Asset parameters
V_0 = 130
phi_V = -3  # * (1.3) # = V0 / L0
sigma_V = 0.05

# Liability parameters
L_0 = 100
phi_L = -3
sigma_L = 0.02

# Interest rate parameters
r_0 = 0.02
k = 0.2
m = 0.05  # 0.5
upsilon = 0.1
eta_r = -0.01

# Catastrophe loss size distribution parameters
mu_C = 2
sigma_C = 0.5

In [3]:
# Poisson process
def simulate_poisson(seed):
    lambda_ = 0.5
    rg = rnd.default_rng(seed)
    return rg.poisson(lambda_ * maturity)


# Cox proces
def simulate_cox(seed):
    lambda0 = 0.49
    a = 0.4
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 0.0, 0.0, 0.5
    )


# Hawkes process
def simulate_hawkes(seed):
    lambda0 = 0.47
    a = 0.26
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.0
    )


# Dynamic contagion process
def simulate_dcp(seed):
    lambda0 = 0.29
    a = 0.26
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.5
    )

## Tables 1-4

In [4]:
%%time
prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    (simulate_poisson, simulate_cox, simulate_hawkes, simulate_dcp),
    mu_C,
    sigma_C,
    markup,
    As=(10.0, 15.0, 20.0, 25.0, 30.0),
    Ms=(60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0),
)

prices.shape

Can't find 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy', generating market conditions
Saving c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy


100%|██████████| 4/4 [01:44<00:00, 26.02s/it]

CPU times: total: 3min 38s
Wall time: 11min 35s


(4, 5, 7)

In [5]:
# Tables 1 to 4
cat_models = ("Poisson", "Cox", "Hawkes", "DCP")
for c in range(4):
    print(cat_models[c])
    display(pd.DataFrame(prices[c]).round(4))

Poisson


,0,1,2,3,4,5,6
0,7.2633,7.2706,7.2742,7.2759,7.2767,7.2771,7.2773
1,4.5877,4.5956,4.5995,4.6013,4.6022,4.6026,4.6028
2,2.7921,2.8008,2.8050,2.8070,2.8079,2.8083,2.8086
3,1.6426,1.6519,1.6565,1.6587,1.6597,1.6602,1.6604
4,0.9380,0.9481,0.9531,0.9554,0.9565,0.9570,0.9573


Cox


,0,1,2,3,4,5,6
0,7.2382,7.2462,7.2502,7.2521,7.2531,7.2536,7.2538
1,4.5959,4.6047,4.6090,4.6110,4.6121,4.6126,4.6129
2,2.8164,2.8260,2.8307,2.8329,2.8340,2.8346,2.8349
3,1.6727,1.6832,1.6883,1.6907,1.6920,1.6925,1.6928
4,0.9650,0.9763,0.9818,0.9845,0.9858,0.9864,0.9868


Hawkes


,0,1,2,3,4,5,6
0,8.0884,8.1620,8.2134,8.2496,8.2754,8.2937,8.3069
1,5.9090,5.9889,6.0443,6.0831,6.1106,6.1301,6.1439
2,4.2961,4.3833,4.4435,4.4853,4.5147,4.5355,4.5502
3,3.0969,3.1919,3.2576,3.3031,3.3349,3.3571,3.3728
4,2.2032,2.3060,2.3775,2.4272,2.4617,2.4858,2.5026


DCP


,0,1,2,3,4,5,6
0,8.1700,8.2422,8.2926,8.3277,8.3521,8.3696,8.3821
1,5.9639,6.0423,6.0967,6.1343,6.1603,6.1789,6.1921
2,4.3290,4.4146,4.4737,4.5142,4.5422,4.5620,4.5760
3,3.1150,3.2083,3.2728,3.3168,3.3470,3.3682,3.3831
4,2.2110,2.3118,2.3820,2.4301,2.4629,2.4857,2.5017


## Table 5: Default risk premium

In [6]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    (simulate_poisson, simulate_cox, simulate_hawkes, simulate_dcp),
    mu_C,
    sigma_C,
    markup,
)[:, np.newaxis]

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:40<00:00, 10.22s/it]


CPU times: total: 9.34 s
Wall time: 47.9 s


In [7]:
risky_prices.round(4)

array([[2.8086],
       [2.8349],
       [4.5502],
       [4.576 ]])

In [8]:
%%time
safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    (simulate_poisson, simulate_cox, simulate_hawkes, simulate_dcp),
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
)[:, np.newaxis]

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:39<00:00,  9.93s/it]


CPU times: total: 9.19 s
Wall time: 46.6 s


In [9]:
safe_prices.round(4)

array([[2.8495],
       [2.879 ],
       [4.8205],
       [4.8414]])

In [10]:
risk_premium = safe_prices - risky_prices
risk_premium.round(4)

array([[0.0409],
       [0.0441],
       [0.2703],
       [0.2654]])

In [11]:
np.hstack((risky_prices, safe_prices, risk_premium)).round(4)

array([[2.8086, 2.8495, 0.0409],
       [2.8349, 2.879 , 0.0441],
       [4.5502, 4.8205, 0.2703],
       [4.576 , 4.8414, 0.2654]])

## Table 6: Impacts of externally-excited jump frequency rate

In [12]:
def simulate_dcp_variations(seed, rho):
    lambda0 = 0.29
    a = 0.26
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.5
    )


simulators = [partial(simulate_dcp_variations, rho=rho) for rho in (0.4, 3, 10, 20)]

In [13]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
).T

Can't find 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy', generating market conditions
Saving c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy


100%|██████████| 4/4 [00:40<00:00, 10.12s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [01:22<00:00, 20.58s/it]


Can't find 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=150,L_0=100,r_0=0.02.npy', generating market conditions
Saving c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=150,L_0=100,r_0=0.02.npy


100%|██████████| 4/4 [00:37<00:00,  9.39s/it]


CPU times: total: 9min 24s
Wall time: 27min 15s


In [14]:
risky_prices.round(4)

array([[ 4.137 ,  4.576 ,  4.7667],
       [13.5344, 15.2673, 16.2062],
       [39.8862, 46.4095, 51.3561],
       [56.6998, 66.8516, 76.0095]])

In [15]:
%%time
safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:35<00:00,  8.76s/it]


CPU times: total: 15 s
Wall time: 42.6 s


In [16]:
safe_prices.round(4)

array([[ 4.8414,  4.8414,  4.8414],
       [16.6552, 16.6552, 16.6552],
       [54.5809, 54.5809, 54.5809],
       [83.1984, 83.1984, 83.1984]])

In [17]:
risk_premium = safe_prices - risky_prices

In [18]:
np.hstack([risky_prices, risk_premium]).round(4)

array([[4.13700e+00, 4.57600e+00, 4.76670e+00, 7.04400e-01, 2.65400e-01,
        7.47000e-02],
       [1.35344e+01, 1.52673e+01, 1.62062e+01, 3.12080e+00, 1.38790e+00,
        4.49000e-01],
       [3.98862e+01, 4.64095e+01, 5.13561e+01, 1.46947e+01, 8.17130e+00,
        3.22480e+00],
       [5.66998e+01, 6.68516e+01, 7.60095e+01, 2.64986e+01, 1.63468e+01,
        7.18890e+00]])

## Table 7: Impacts of externally-excited jump magnitude

In [19]:
def simulate_dcp_variations(seed, mu_F=0.25, mu_G=0.5):
    lambda0 = 0.29
    a = 0.26
    delta = 1
    rho = 0.4

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 2 * mu_G, 0.0, 2 * mu_F
    )


simulators = [
    partial(simulate_dcp_variations, mu_F=mu_F) for mu_F in (0.25, 1.0, 4.0, 8.0)
]

In [20]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
).T

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

risk_premium = safe_prices - risky_prices

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:29<00:00,  7.40s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:27<00:00,  6.97s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=150,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:28<00:00,  7.00s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:26<00:00,  6.69s/it]

CPU times: total: 41.2 s
Wall time: 2min 21s


In [21]:
np.hstack([risky_prices, risk_premium]).round(4)

array([[ 4.137 ,  4.576 ,  4.7667,  0.7044,  0.2654,  0.0747],
       [ 8.5331,  9.5955, 10.1627,  1.9013,  0.8389,  0.2717],
       [23.304 , 27.0421, 29.9316,  8.6188,  4.8807,  1.9912],
       [31.4876, 36.8698, 41.4919, 13.5394,  8.1572,  3.5351]])

## Table 8: Impacts of self-excited jump magnitude

In [22]:
%%time

simulators = [
    partial(simulate_dcp_variations, mu_G=mu_G) for mu_G in (0.5, 1.0, 2.0, 3.0)
]

risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
).T

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

risk_premium = safe_prices - risky_prices

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:34<00:00,  8.55s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:34<00:00,  8.62s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=150,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:34<00:00,  8.53s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=1000000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:33<00:00,  8.27s/it]

CPU times: total: 1min 5s
Wall time: 2min 45s


In [23]:
np.hstack([risky_prices, risk_premium]).round(4)

array([[ 4.137 ,  4.576 ,  4.7667,  0.7044,  0.2654,  0.0747],
       [ 9.145 , 10.4484, 11.3248,  2.7225,  1.4191,  0.5427],
       [19.0516, 22.2658, 25.0118,  8.0721,  4.858 ,  2.112 ],
       [24.8796, 29.2425, 33.1769, 11.4628,  7.0999,  3.1656]])